In [1]:
import sys
sys.path.append('../../30_data_tools/')
from pathlib import Path
from random import choices
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import numpy as np
import pandas as pd
import torch
from file_interaction import download_blob, get_related_filepath, open_img
from tqdm.auto import tqdm
import sqlite3
import re
import cv2
# import YOLO model
from ultralytics import YOLO
from io import BytesIO
from torchvision.transforms.functional import pil_to_tensor

In [2]:
from helper import load_dotenv
from get_labelstudio_data import get_results_of_project
from classification_tools import cut_img_into_tiles, preclassifier, classifier

In [3]:
Image.MAX_IMAGE_PIXELS = None

In [4]:
TILE_SIZE = 224
MIN_MIDTONE_SHARE = 0.05
MARGIN = 0.03
MAX_BATCH_SIZE = 64
RESOLUTIONS = [300,200,150]

In [5]:
model_name = '2024-05-01_Resnet50_004'

In [6]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [7]:
exclude_jobs = [
    '23-10-03_Testformen',
    '23-10-18_farbe',
    '23-10-17_blur',
    '23-10-19_farbe'
]

include_jobs = [
    '24-03-05-01_randomTrainPages'
]

In [8]:
results = [
    r for r in get_results_of_project(3)
    if r['labels'][0] not in ['potential_moire'] and True not in [r['img_name'].startswith(ej) for ej in exclude_jobs]
]

In [9]:
pages = pd.read_sql(
    f'''
        SELECT rf.* FROM related_file rf 
        LEFT JOIN (SELECT job, pdf_filename, 1 AS used_as_base FROM generic_image gi ) gi
        ON rf.job=gi.job AND rf.pdf_filename = gi.pdf_filename
        WHERE variant_name = 'ps2400dpi150lpi' AND "type" = '4c_{ dotenv["LOFI_DPI"] }' AND gi.used_as_base IS NULL
    ''',
    con
)

In [10]:
bytesStream = download_blob( f'models/{ model_name }.pth' )
model = torch.load( BytesIO(bytesStream.getvalue()), map_location=torch.device('cpu') )

# Funktionen

In [11]:
def get_out_map( img, relevant_tiles ):
    out_img = np.zeros((img.size[1], img.size[0], 4))
    colors = [
        (0,0,255),
        (255,0,0)
    ]
    
    for pos,tile,pred,label in relevant_tiles:
        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            0
        ] += 1

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            1
        ] += float(pred[1])

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            2
        ] += float(pred[0])

    out_img /= 2

    level_1_img = Image.fromarray((out_img[:,:,0] * 255).astype('uint8'))
    level_2_img = Image.fromarray((out_img[:,:,1] * 255).astype('uint8'))
    level_3_img = Image.fromarray((out_img[:,:,2] * 255).astype('uint8'))
    level_4_img = out_img[:,:,2] * 255
    level_4_img[(out_img[:,:,2] < out_img[:,:,1]) | (out_img[:,:,2] < 0.5)] = 0
    level_4_img = Image.fromarray(level_4_img.astype('uint8'))
    
    return level_1_img, level_2_img, level_3_img, level_4_img

In [52]:
def create_result_map( img_size, tiles ):
    result_map = np.zeros((img_size[1],img_size[0])).astype('float32')
    
    for dpi,pos,tile,pred,label in tiles:
        rescale_factor = dotenv['LOFI_DPI'] / dpi
        pos = (
            round(rescale_factor * pos[0]),
            round(rescale_factor * pos[1])
        )
        tile_size = round(rescale_factor * TILE_SIZE)
        
        if pred[1] < pred[0]:
            result_map[
                pos[1]:pos[1]+tile_size,
                pos[0]:pos[0]+tile_size,
            ] += 1

    return result_map

In [13]:
def get_moires_of_page( row, results ):
    relevant_moires = []
    
    for r in results:
        if re.match(f'^{ row.job }\..+?\.{ row.pdf_filename }\..+', r['img_name']):
            dpi = int( re.match(r'.+\.4c_(\d+)\.jpg', r['img_name']).groups()[0] )
            out_box = [r['value']['x'],r['value']['y'],r['value']['width'],r['value']['height']]
            
            # box umrechnen
            if target_dpi != dpi:
                out_box = [round(val * (target_dpi / dpi)) for val in out_box]
    
            relevant_moires.append(out_box)

    return relevant_moires

In [14]:
def get_intersection_over_union( box_a, box_b ):
    intersection_box = [
        max(box_a[0],box_b[0]),
        max(box_a[1],box_b[1]),
        min(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        min(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]

    if intersection_box[2] - intersection_box[0] < 0 or intersection_box[3] - intersection_box[1] < 0:
        return 0

    intersection = (intersection_box[2] - intersection_box[0]) * (intersection_box[3] - intersection_box[1])
    
    union_box = [
        min(box_a[0],box_b[0]),
        min(box_a[1],box_b[1]),
        max(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        max(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]
    union = (union_box[2] - union_box[0]) * (union_box[3] - union_box[1])

    return intersection / union

In [58]:
def get_result_boxes( result_map, threshold=1 ):
    thresh = np.zeros(result_map.shape).astype('uint8')
    thresh[result_map > threshold] = 255

    (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(
    	thresh, 4, cv2.CV_32S
    )

    return [
        [b[0],b[1],b[2],b[3]]
        for b in stats[1:]
    ]

In [16]:
def draw_bounding_boxes( img, moire_boxes, predicted_boxes ):
    colors = {
        "target" : "green",
        "predicted" : "red"
    }    

    img_out = img.copy().convert('RGB')
    draw = ImageDraw.Draw(img_out) 

    for b in moire_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['target'], width=10) 

    for b in predicted_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['predicted'], width=10) 

    
    return img_out

In [17]:
0 / 0

ZeroDivisionError: division by zero

# Auswertung

In [18]:
samples = pages.sample(n=10)

In [77]:
moire_images = list(set([r['img_name'] for r in get_results_of_project(3) if r['labels'][0] == 'moire']))
moires = {}

for r in get_results_of_project(3):
    if r['labels'][0] == 'moire':
        moire_key = re.match(r'(.+?)\.(v?ps2400dpi\d+lpi)\.(.+).(4c_\d00).jpg', r['img_name']).groups()

        if moire_key not in moires:
            moires[moire_key] = []

        moires[moire_key].append(r)

In [20]:
moire_pages = pd.DataFrame(
    [
        re.match(r'(.+?)\.(v?ps2400dpi\d+lpi)\.(.+).(4c_\d00).jpg', m).groups()
        for m in moire_images
    ],
    columns=['job','variant_name','pdf_filename','type']
)

In [21]:
samples = pd.merge(
    moire_pages,
    pages,
    how="left",
    on=['job','pdf_filename','variant_name','type']
)
samples = samples.loc[pd.isna(samples.filename) == False]

In [22]:
CREATE_BLEND_IMAGE = False
CREATE_RESULT_BOXES_IMAGE = True

In [107]:
for i in tqdm(range(samples.shape[0])):
    sample = samples.iloc[i]
    
    img_path = get_related_filepath(
        sample.job,
        'ps2400dpi150lpi',
        f'{ sample.pdf_filename }.4c_{ dotenv["LOFI_DPI"] }.jpg'
    )
    img = open_img(img_path)

    tiles = cut_img_into_tiles( img, dotenv["LOFI_DPI"], RESOLUTIONS )
    tiles_preclassified = preclassifier( tiles )
    predictions = classifier( tiles_preclassified, model, tile_size=TILE_SIZE, max_batch_size=MAX_BATCH_SIZE )
    
    result_map = create_result_map( img.size, predictions )    
    predicted_label = int(result_map[result_map > 0.5].shape[0] > 0)

    if CREATE_RESULT_BOXES_IMAGE:
        result_boxes = get_result_boxes( result_map )
        moire_key = (sample.job,sample.variant_name,sample.pdf_filename,sample.type)
        
        if moire_key in moires:
            moire_boxes = []
        
            for m in moires[moire_key]:
                min_x = int(round(min([pt[0] / 100 * img.size[0] for pt in m['value']['points']])))
                min_y = int(round(min([pt[1] / 100 * img.size[1] for pt in m['value']['points']])))
                
                max_x = int(round(max([pt[0] / 100 * img.size[0] for pt in m['value']['points']])))
                max_y = int(round(max([pt[1] / 100 * img.size[1] for pt in m['value']['points']])))
            
                moire_boxes.append([min_x, min_y, max_x - min_x, max_y - min_y])
        else:
            moire_boxes = []
        
        box_img = draw_bounding_boxes( img, moire_boxes, result_boxes )
        box_img.save( dotenv['TEMP_PROCESSING_DIR'] / 'boxes_page_results' / f"boxes.{ sample.job }.{ sample.pdf_filename }.{ predicted_label }.jpg", progressive=True )
    
    if CREATE_BLEND_IMAGE:
        # blend img erzeugen
        TARGET_OUT_HEIGHT = 1000
        colors=['red','green','blue','orange']
        l_images = get_out_map( img, predictions )
        blended = Image.new(mode="RGB", size=(img.size[0] * len(l_images), img.size[1]))
    
        for i in range(len(l_images)):
            l_img = ImageEnhance.Brightness(l_images[i]).enhance(0.5)
            overlay = Image.new('RGB', l_img.size, color=colors[i])
            l_rgb = img.convert('RGB')
            l_rgb.paste(
                overlay,
                (0,0),
                mask=l_img
            )
    
            blended.paste(
                l_rgb,
                (img.size[0] * i,0)
            )
    
        #blended = blended.resize((
        #    round(TARGET_OUT_HEIGHT / blended.size[1] * blended.size[0]),
        #    TARGET_OUT_HEIGHT
        #))
        
        blended.save( dotenv['TEMP_PROCESSING_DIR'] / 'blended_page_results' / f"separated.{ sample.job }.{ sample.pdf_filename }.{ predicted_label }.jpg", progressive=True )

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/293 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/348 [00:00<?, ?it/s]

  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]